# Introduction

The goal of this case study is to read in the Fantasy Football information we have gathered in order to determine the best player at certain positions every year (1999 - 2019) in the NFL.

Our strategy is to use Fantasy Points as a basis for individual performance. However, a major part will be instead of just judging players soley on how many points they gain in a season we will also be looking at how good their opponent's defense was against their position.

Firstly we will read in the fantasy data, cleaning it and adding the different opponents to the data from a schedule file. We will then remove the outliers from the data before finding average performance by position through the years. We will find the average using EMA. We will then find the average yearly performance of each team's defense against the 5 positions we are tracking for each individual year. To normalize our data we will be taking the difference between the opponent team's average fantasy points given up and the positons EMA for that year. That difference will give us our Adjusted Points Value. A players Adjusted Point value will be totaled for the entire season, the player with the highest total Adjusted Points for the year at their position will be determined the top preformer.


# Initial Library Import

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

# Data Collection & Cleaning
##Schedule Import & Adjustment
The NFL Schedule for the 1999 - 2019 Seasons was originally scraped from <a href="https://www.pro-football-reference.com/" target="_blank">https://www.pro-football-reference.com/</a> . The Schedule uses the teams full name and needs to be converted into the abbrivation that is used in the Fantasy Data Set.


In [ ]:
url = 'https://raw.githubusercontent.com/JamesDenbow/FantasyFootballDataSet/main/schedule.csv'
scheduleDF = pd.read_csv(url)

scheduleDF = scheduleDF.replace(['Arizona Cardinals'], 'ARI')
scheduleDF = scheduleDF.replace(['Atlanta Falcons'], 'ATL')
scheduleDF = scheduleDF.replace(['Carolina Panthers'], 'CAR')
scheduleDF = scheduleDF.replace(['Chicago Bears'], 'CHI')
scheduleDF = scheduleDF.replace(['Dallas Cowboys'], 'DAL')
scheduleDF = scheduleDF.replace(['Detroit Lions'], 'DET')
scheduleDF = scheduleDF.replace(['Green Bay Packers'], 'GNB')
scheduleDF = scheduleDF.replace(['St. Louis Rams', 'Los Angeles Rams'], 'STL')
scheduleDF = scheduleDF.replace(['Minnesota Vikings'], 'MIN')
scheduleDF = scheduleDF.replace(['New Orleans Saints'], 'NOR')
scheduleDF = scheduleDF.replace(['New York Giants'], 'NYG')
scheduleDF = scheduleDF.replace(['Philadelphia Eagles'], 'PHI')
scheduleDF = scheduleDF.replace(['San Francisco 49ers'], 'SFO')
scheduleDF = scheduleDF.replace(['Seattle Seahawks'], 'SEA')
scheduleDF = scheduleDF.replace(['Tampa Bay Buccaneers'], 'TAM')
scheduleDF = scheduleDF.replace(['Washington Redskins'], 'WAS')

scheduleDF = scheduleDF.replace(['Baltimore Ravens'], 'BAL')
scheduleDF = scheduleDF.replace(['Buffalo Bills'], 'BUF')
scheduleDF = scheduleDF.replace(['Cincinnati Bengals'], 'CIN')
scheduleDF = scheduleDF.replace(['Cleveland Browns'], 'CLE')
scheduleDF = scheduleDF.replace(['Denver Broncos'], 'DEN')
scheduleDF = scheduleDF.replace(['Houston Texans'], 'HOU')
scheduleDF = scheduleDF.replace(['Indianapolis Colts'], 'IND')
scheduleDF = scheduleDF.replace(['Jacksonville Jaguars'], 'JAX')
scheduleDF = scheduleDF.replace(['Kansas City Chiefs'], 'KAN')
scheduleDF = scheduleDF.replace(['Oakland Raiders'], 'OAK')
scheduleDF = scheduleDF.replace(['San Diego Chargers', 'Los Angeles Chargers'], 'SDG')
scheduleDF = scheduleDF.replace(['Miami Dolphins'], 'MIA')
scheduleDF = scheduleDF.replace(['New England Patriots'], 'NWE')
scheduleDF = scheduleDF.replace(['New York Jets'], 'NYJ')
scheduleDF = scheduleDF.replace(['Pittsburgh Steelers'], 'PIT')
scheduleDF = scheduleDF.replace(['Tennessee Titans'], 'TEN')


print (scheduleDF)

     GameTime Winner Loser
0      1999-1    DAL   WAS
1      1999-1    NWE   NYJ
2      1999-1    CHI   KAN
3      1999-1    NOR   CAR
4      1999-1    STL   BAL
...       ...    ...   ...
5347  2019-17    PHI   NYG
5348  2019-17    DAL   WAS
5349  2019-17    BAL   PIT
5350  2019-17    STL   ARI
5351  2019-17    SFO   SEA

[5352 rows x 3 columns]


## Clean Fantasy Data

The fantasy data was taken from <a href="https://www.fantasyfootballdatapros.com/csv_files" target="_blank">https://www.fantasyfootballdatapros.com/csv_files</a> . Here we are doing some small cleaning of the data and loading into a dataframe.

In [ ]:
completeDF = pd.DataFrame()
for i in range(1999, 2020):
  for x in range(1, 18):
    url = 'https://raw.githubusercontent.com/JamesDenbow/FantasyFootballDataSet/main/' + str(i) + '/week' + str(x) + '.csv'
    df = pd.read_csv(url)
    #Removes unneeded columns
    df = df.drop(columns=['PPRFantasyPoints', 'HalfPPRFantasyPoints'])
    #Remove players who did not have any fantasy points
    df = df[df.StandardFantasyPoints > 0]
    df['Year'] = str(i)
    df['Week'] = str(x)
    #Converts the San Diego Chargers and Los Angeles Chargers into a standard abbreviation
    df = df.replace(['LAC'], 'SDG')
    #Converts the St. Louis Rams and Los Angeles Rams into a standard abbreviation
    df = df.replace(['LAR'], 'STL')
    #Creates empty column for opponents
    df['Opp'] = ''
    #Searches ScheduleDF to find the opponents for each team
    gametime = str(i) + '-' + str(x)
    tempScheduleDF = scheduleDF[scheduleDF.GameTime == gametime]
    for row in df.index:
      team = df['Tm'][row]
      if tempScheduleDF.isin([team]).any().Winner:
        temp = tempScheduleDF.loc[tempScheduleDF['Winner'] == team]
        df.at[row,'Opp'] = str(temp.iloc[0]['Loser'])
      elif tempScheduleDF.isin([team]).any().Loser:
        temp = tempScheduleDF.loc[tempScheduleDF['Loser'] == team]
        df.at[row,'Opp'] = str(temp.iloc[0]['Winner'])
    completeDF = completeDF.append(df, ignore_index=True)

#Cut down the positions in the data
completeDF = completeDF.replace(['WR/RS'], 'WR')
completeDF = completeDF.replace(['QB/P'], 'QB')
completeDF = completeDF.replace(['FB/TE'], 'FB')
completeDF = completeDF.replace(['FB/RB'], 'FB')
completeDF = completeDF.replace(['WR/K'], 'WR')
completeDF = completeDF.replace(['SE'], 'TE')
completeDF = completeDF.replace(['FL'], 'WR')
completeDF = completeDF.replace(['XR'], 'WR')
completeDF = completeDF.replace(['ZR'], 'WR')
completeDF = completeDF.replace(['FB/DL'], 'FB')
completeDF = completeDF.replace(['WR/PR'], 'WR')
completeDF = completeDF.replace(['RB/K'], 'HB')
completeDF = completeDF.replace(['RB'], 'HB')
completeDF = completeDF.replace(['TB'], 'TE')
completeDF = completeDF.replace(['WC'], 'WR')
completeDF = completeDF.replace(['WR/P'], 'WR')
completeDF = completeDF.replace(['Z'], 'WR')
completeDF = completeDF.replace(['H-B'], 'HB')
completeDF = completeDF.replace(['WR/T'], 'WR')
completeDF = completeDF.replace(['WR-K'], 'WR')
completeDF = completeDF.replace(['FB/R'], 'FB')
completeDF = completeDF.replace(['WR/R'], 'WR')
completeDF = completeDF.replace(['TE/F'], 'TE')
completeDF = completeDF.replace(['FB/T'], 'FB')
completeDF = completeDF.replace(['OW'], 'HB')
completeDF = completeDF.replace(['F'], 'TE')
completeDF = completeDF.replace(['KR/W'], 'HB')
completeDF = completeDF.replace(['TE-F'], 'HB')
completeDF = completeDF.replace(['X'], 'WR')
completeDF = completeDF.replace(['WR-R'], 'WR')
completeDF = completeDF.replace(['WR W'], 'WR')
completeDF = completeDF.replace(['RB/F'], 'HB')
completeDF = completeDF.replace(['TE/W'], 'TE')
completeDF = completeDF.replace(['QB-W'], 'WR')
completeDF = completeDF.replace(['QB/W'], 'WR')
completeDF = completeDF.replace(['FB-T'], 'FB')
completeDF = completeDF.replace(['HB-K'], 'HB')
completeDF = completeDF.replace(['RB-K'], 'HB')
completeDF = completeDF.replace(['TE/L'], 'TE')
completeDF = completeDF.replace(['HB/F'], 'HB')
completeDF = completeDF.replace(['XTE'], 'TE')
completeDF = completeDF.replace(['LWR'], 'WR')
completeDF = completeDF.replace(['RWR'], 'WR')
completeDF = completeDF.replace(["FB'"], 'FB')
completeDF = completeDF.replace(['TE-L'], 'TE')
completeDF = completeDF.replace(['P-QB'], 'QB')
completeDF = completeDF.replace(['PR/W'], 'HB')
completeDF = completeDF.replace(['XWR'], 'WR')
completeDF = completeDF.replace(['WR-P'], 'WR')
completeDF = completeDF.replace(['RB-F'], 'HB')
completeDF = completeDF.replace(['FB-R'], 'FB')
completeDF = completeDF.replace(['HB/K'], 'HB')
completeDF = completeDF.replace(['3QB'], 'QB')
completeDF = completeDF.replace(['RB-KR'], 'HB')
completeDF = completeDF.replace(['RB/KR'], 'HB')
completeDF = completeDF.replace(['X-WR'], 'WR')
completeDF = completeDF.replace(['WR-KR'], 'WR')
completeDF = completeDF.replace(['QB3'], 'QB')
completeDF = completeDF.replace(['QB-WR'], 'WR')
completeDF = completeDF.replace(['QB/WR'], 'WR')
completeDF = completeDF.replace(['#3 QB'], 'QB')
completeDF = completeDF.replace(['TE/FB'], 'FB')
completeDF = completeDF.replace(['TR'], 'TE')
completeDF = completeDF.replace(['KR/H'], 'WR')

#Remove positions that are not going to be tracked
completeDF = completeDF[completeDF.Pos != 'DT']
completeDF = completeDF[completeDF.Pos != 'S']
completeDF = completeDF[completeDF.Pos != 'T']
completeDF = completeDF[completeDF.Pos != 'P']
completeDF = completeDF[completeDF.Pos != 'LB']
completeDF = completeDF[completeDF.Pos != 'DB/LB']
completeDF = completeDF[completeDF.Pos != 'DL']
completeDF = completeDF[completeDF.Pos != 'T/G']
completeDF = completeDF[completeDF.Pos != 'C']
completeDF = completeDF[completeDF.Pos != 'OL']
completeDF = completeDF[completeDF.Pos != 'CB']
completeDF = completeDF[completeDF.Pos != 'FS']
completeDF = completeDF[completeDF.Pos != 'KR']
completeDF = completeDF[completeDF.Pos != 'ILB']
completeDF = completeDF[completeDF.Pos != 'SS']
completeDF = completeDF[completeDF.Pos != 'LT']
completeDF = completeDF[completeDF.Pos != 'RDT']
completeDF = completeDF[completeDF.Pos != 'RCB']
completeDF = completeDF[completeDF.Pos != 'DB']
completeDF = completeDF[completeDF.Pos != 'RT']
completeDF = completeDF[completeDF.Pos != 'RG']
completeDF = completeDF[completeDF.Pos != 'LDT']
completeDF = completeDF[completeDF.Pos != 'DE']
completeDF = completeDF[completeDF.Pos != 'NT']
completeDF = completeDF[completeDF.Pos != 'LG']
completeDF = completeDF[completeDF.Pos != 'WLB']
completeDF = completeDF[completeDF.Pos != 'LOLB']
completeDF = completeDF[completeDF.Pos != 'RDE']
completeDF = completeDF[completeDF.Pos != 'LS']
completeDF = completeDF[completeDF.Pos != 'LCB']
completeDF = completeDF[completeDF.Pos != 'CB/K']
completeDF = completeDF[completeDF.Pos != 'KR-R']
completeDF = completeDF[completeDF.Pos != 'G']
completeDF = completeDF[completeDF.Pos != 'OT']
completeDF = completeDF[completeDF.Pos != 'LOT']
completeDF = completeDF[completeDF.Pos != 'MLB']
completeDF = completeDF[completeDF.Pos != 'PK']
completeDF = completeDF[completeDF.Pos != 'P/K']
completeDF = completeDF[completeDF.Pos != 'KR/P']
completeDF = completeDF[completeDF.Pos != 'OLB']
completeDF = completeDF[completeDF.Pos != 'G/T']
completeDF = completeDF[completeDF.Pos != 'PR']
completeDF = completeDF[completeDF.Pos != 'DL/L']
completeDF = completeDF[completeDF.Pos != 'CB/W']
completeDF = completeDF[completeDF.Pos != '24']
completeDF = completeDF[completeDF.Pos != '0']

completeDF = completeDF[completeDF.Pos != 'K']

completeDF = completeDF.dropna()

print (completeDF)

              Player Pos   Tm  ...  Year  Week  Opp
0       Mike Tomczak  QB  PIT  ...  1999     1  CLE
1           Tom Tupa  QB  NYJ  ...  1999     1  NWE
2         Dan Marino  QB  MIA  ...  1999     1  DEN
3       Brad Johnson  QB  WAS  ...  1999     1  DAL
4       Steve McNair  QB  TEN  ...  1999     1  CIN
...              ...  ..  ...  ...   ...   ...  ...
93119  Cyril Grayson  WR  TAM  ...  2019    17  ATL
93120  Cameron Brate  TE  TAM  ...  2019    17  ATL
93121   Johnny Mundt  TE  STL  ...  2019    17  ARI
93122   Tavon Austin  WR  DAL  ...  2019    17  WAS
93123     Blake Bell  TE  KAN  ...  2019    17  SDG

[92540 rows x 20 columns]


# Calculating Averages
##Calculate General Yearly Averages by Position

In [ ]:
#Seperate by years
df99 = completeDF[completeDF.Year == '1999']
df00 = completeDF[completeDF.Year == '2000']
df01 = completeDF[completeDF.Year == '2001']
df02 = completeDF[completeDF.Year == '2002']
df03 = completeDF[completeDF.Year == '2003']
df04 = completeDF[completeDF.Year == '2004']
df05 = completeDF[completeDF.Year == '2005']
df06 = completeDF[completeDF.Year == '2006']
df07 = completeDF[completeDF.Year == '2007']
df08 = completeDF[completeDF.Year == '2008']
df09 = completeDF[completeDF.Year == '2009']
df10 = completeDF[completeDF.Year == '2010']
df11 = completeDF[completeDF.Year == '2011']
df12 = completeDF[completeDF.Year == '2012']
df13 = completeDF[completeDF.Year == '2013']
df14 = completeDF[completeDF.Year == '2014']
df15 = completeDF[completeDF.Year == '2015']
df16 = completeDF[completeDF.Year == '2016']
df17 = completeDF[completeDF.Year == '2017']
df18 = completeDF[completeDF.Year == '2018']
df19 = completeDF[completeDF.Year == '2019']

#Seperate by position
df99qb = df99.loc[df99.Pos == 'QB']
df99wr = df99.loc[df99.Pos == 'WR']
df99te = df99.loc[df99.Pos == 'TE']
df99fb = df99.loc[df99.Pos == 'FB']
df99hb = df99.loc[df99.Pos == 'HB']

df00qb = df00.loc[df00.Pos == 'QB']
df00wr = df00.loc[df00.Pos == 'WR']
df00te = df00.loc[df00.Pos == 'TE']
df00fb = df00.loc[df00.Pos == 'FB']
df00hb = df00.loc[df00.Pos == 'HB']

df01qb = df01.loc[df01.Pos == 'QB']
df01wr = df01.loc[df01.Pos == 'WR']
df01te = df01.loc[df01.Pos == 'TE']
df01fb = df01.loc[df01.Pos == 'FB']
df01hb = df01.loc[df01.Pos == 'HB']

df02qb = df02.loc[df02.Pos == 'QB']
df02wr = df02.loc[df02.Pos == 'WR']
df02te = df02.loc[df02.Pos == 'TE']
df02fb = df02.loc[df02.Pos == 'FB']
df02hb = df02.loc[df02.Pos == 'HB']

df03qb = df03.loc[df03.Pos == 'QB']
df03wr = df03.loc[df03.Pos == 'WR']
df03te = df03.loc[df03.Pos == 'TE']
df03fb = df03.loc[df03.Pos == 'FB']
df03hb = df03.loc[df03.Pos == 'HB']

df04qb = df04.loc[df04.Pos == 'QB']
df04wr = df04.loc[df04.Pos == 'WR']
df04te = df04.loc[df04.Pos == 'TE']
df04fb = df04.loc[df04.Pos == 'FB']
df04hb = df04.loc[df04.Pos == 'HB']

df05qb = df05.loc[df05.Pos == 'QB']
df05wr = df05.loc[df05.Pos == 'WR']
df05te = df05.loc[df05.Pos == 'TE']
df05fb = df05.loc[df05.Pos == 'FB']
df05hb = df05.loc[df05.Pos == 'HB']

df06qb = df06.loc[df06.Pos == 'QB']
df06wr = df06.loc[df06.Pos == 'WR']
df06te = df06.loc[df06.Pos == 'TE']
df06fb = df06.loc[df06.Pos == 'FB']
df06hb = df06.loc[df06.Pos == 'HB']

df07qb = df07.loc[df07.Pos == 'QB']
df07wr = df07.loc[df07.Pos == 'WR']
df07te = df07.loc[df07.Pos == 'TE']
df07fb = df07.loc[df07.Pos == 'FB']
df07hb = df07.loc[df07.Pos == 'HB']

df08qb = df08.loc[df08.Pos == 'QB']
df08wr = df08.loc[df08.Pos == 'WR']
df08te = df08.loc[df08.Pos == 'TE']
df08fb = df08.loc[df08.Pos == 'FB']
df08hb = df08.loc[df08.Pos == 'HB']

df09qb = df09.loc[df09.Pos == 'QB']
df09wr = df09.loc[df09.Pos == 'WR']
df09te = df09.loc[df09.Pos == 'TE']
df09fb = df09.loc[df09.Pos == 'FB']
df09hb = df09.loc[df09.Pos == 'HB']

df10qb = df10.loc[df10.Pos == 'QB']
df10wr = df10.loc[df10.Pos == 'WR']
df10te = df10.loc[df10.Pos == 'TE']
df10fb = df10.loc[df10.Pos == 'FB']
df10hb = df10.loc[df10.Pos == 'HB']

df11qb = df11.loc[df11.Pos == 'QB']
df11wr = df11.loc[df11.Pos == 'WR']
df11te = df11.loc[df11.Pos == 'TE']
df11fb = df11.loc[df11.Pos == 'FB']
df11hb = df11.loc[df11.Pos == 'HB']

df12qb = df12.loc[df12.Pos == 'QB']
df12wr = df12.loc[df12.Pos == 'WR']
df12te = df12.loc[df12.Pos == 'TE']
df12fb = df12.loc[df12.Pos == 'FB']
df12hb = df12.loc[df12.Pos == 'HB']

df13qb = df13.loc[df13.Pos == 'QB']
df13wr = df13.loc[df13.Pos == 'WR']
df13te = df13.loc[df13.Pos == 'TE']
df13fb = df13.loc[df13.Pos == 'FB']
df13hb = df13.loc[df13.Pos == 'HB']

df14qb = df14.loc[df14.Pos == 'QB']
df14wr = df14.loc[df14.Pos == 'WR']
df14te = df14.loc[df14.Pos == 'TE']
df14fb = df14.loc[df14.Pos == 'FB']
df14hb = df14.loc[df14.Pos == 'HB']

df15qb = df15.loc[df15.Pos == 'QB']
df15wr = df15.loc[df15.Pos == 'WR']
df15te = df15.loc[df15.Pos == 'TE']
df15fb = df15.loc[df15.Pos == 'FB']
df15hb = df15.loc[df15.Pos == 'HB']

df16qb = df16.loc[df16.Pos == 'QB']
df16wr = df16.loc[df16.Pos == 'WR']
df16te = df16.loc[df16.Pos == 'TE']
df16fb = df16.loc[df16.Pos == 'FB']
df16hb = df16.loc[df16.Pos == 'HB']

df17qb = df17.loc[df17.Pos == 'QB']
df17wr = df17.loc[df17.Pos == 'WR']
df17te = df17.loc[df17.Pos == 'TE']
df17fb = df17.loc[df17.Pos == 'FB']
df17hb = df17.loc[df17.Pos == 'HB']

df18qb = df18.loc[df18.Pos == 'QB']
df18wr = df18.loc[df18.Pos == 'WR']
df18te = df18.loc[df18.Pos == 'TE']
df18fb = df18.loc[df18.Pos == 'FB']
df18hb = df18.loc[df18.Pos == 'HB']

df19qb = df19.loc[df19.Pos == 'QB']
df19wr = df19.loc[df19.Pos == 'WR']
df19te = df19.loc[df19.Pos == 'TE']
df19fb = df19.loc[df19.Pos == 'FB']
df19hb = df19.loc[df19.Pos == 'HB']

## Calculate Z-Scores
Calculate the Z-score by each years position group. A Z-Score is a tangable evaluation of how far from the mean a data point is. A performance with a Z-Score higher then 3.0 or less then -3.0 is determined to be an outlier and removed from the calculations.

In [ ]:
def find_z(df):
  mean = df['StandardFantasyPoints'].mean()
  std = df['StandardFantasyPoints'].std()
  df['Z'] = (df['StandardFantasyPoints'] - mean) / std
  #Remove Outliers
  df = df[df.Z < 3]
  df = df[df.Z > -3]
  return df

#Remove Outliers
df99qb = find_z(df99qb)
df99wr = find_z(df99wr)
df99te = find_z(df99te)
df99fb = find_z(df99fb)
df99hb = find_z(df99hb)

df00qb = find_z(df00qb)
df00wr = find_z(df00wr)
df00te = find_z(df00te)
df00fb = find_z(df00fb)
df00hb = find_z(df00hb)

df01qb = find_z(df01qb)
df01wr = find_z(df01wr)
df01te = find_z(df01te)
df01fb = find_z(df01fb)
df01hb = find_z(df01hb)

df02qb = find_z(df02qb)
df02wr = find_z(df02wr)
df02te = find_z(df02te)
df02fb = find_z(df02fb)
df02hb = find_z(df02hb)

df03qb = find_z(df03qb)
df03wr = find_z(df03wr)
df03te = find_z(df03te)
df03fb = find_z(df03fb)
df03hb = find_z(df03hb)

df04qb = find_z(df04qb)
df04wr = find_z(df04wr)
df04te = find_z(df04te)
df04fb = find_z(df04fb)
df04hb = find_z(df04hb)

df05qb = find_z(df05qb)
df05wr = find_z(df05wr)
df05te = find_z(df05te)
df05fb = find_z(df05fb)
df05hb = find_z(df05hb)

df06qb = find_z(df06qb)
df06wr = find_z(df06wr)
df06te = find_z(df06te)
df06fb = find_z(df06fb)
df06hb = find_z(df06hb)

df07qb = find_z(df07qb)
df07wr = find_z(df07wr)
df07te = find_z(df07te)
df07fb = find_z(df07fb)
df07hb = find_z(df07hb)

df08qb = find_z(df08qb)
df08wr = find_z(df08wr)
df08te = find_z(df08te)
df08fb = find_z(df08fb)
df08hb = find_z(df08hb)

df09qb = find_z(df09qb)
df09wr = find_z(df09wr)
df09te = find_z(df09te)
df09fb = find_z(df09fb)
df09hb = find_z(df09hb)

df10qb = find_z(df10qb)
df10wr = find_z(df10wr)
df10te = find_z(df10te)
df10fb = find_z(df10fb)
df10hb = find_z(df10hb)

df11qb = find_z(df11qb)
df11wr = find_z(df11wr)
df11te = find_z(df11te)
df11fb = find_z(df11fb)
df11hb = find_z(df11hb)

df12qb = find_z(df12qb)
df12wr = find_z(df12wr)
df12te = find_z(df12te)
df12fb = find_z(df12fb)
df12hb = find_z(df12hb)

df13qb = find_z(df13qb)
df13wr = find_z(df13wr)
df13te = find_z(df13te)
df13fb = find_z(df13fb)
df13hb = find_z(df13hb)

df14qb = find_z(df14qb)
df14wr = find_z(df14wr)
df14te = find_z(df14te)
df14fb = find_z(df14fb)
df14hb = find_z(df14hb)

df15qb = find_z(df15qb)
df15wr = find_z(df15wr)
df15te = find_z(df15te)
df15fb = find_z(df15fb)
df15hb = find_z(df15hb)

df16qb = find_z(df16qb)
df16wr = find_z(df16wr)
df16te = find_z(df16te)
df16fb = find_z(df16fb)
df16hb = find_z(df16hb)

df17qb = find_z(df17qb)
df17wr = find_z(df17wr)
df17te = find_z(df17te)
df17fb = find_z(df17fb)
df17hb = find_z(df17hb)

df18qb = find_z(df18qb)
df18wr = find_z(df18wr)
df18te = find_z(df18te)
df18fb = find_z(df18fb)
df18hb = find_z(df18hb)

df19qb = find_z(df19qb)
df19wr = find_z(df19wr)
df19te = find_z(df19te)
df19fb = find_z(df19fb)
df19hb = find_z(df19hb)

#Find yearly averages by position
avg_99qb = df99qb['StandardFantasyPoints'].mean()
avg_99wr = df99wr['StandardFantasyPoints'].mean()
avg_99te = df99te['StandardFantasyPoints'].mean()
avg_99fb = df99fb['StandardFantasyPoints'].mean()
avg_99hb = df99hb['StandardFantasyPoints'].mean()

avg_00qb = df00qb['StandardFantasyPoints'].mean()
avg_00wr = df00wr['StandardFantasyPoints'].mean()
avg_00te = df00te['StandardFantasyPoints'].mean()
avg_00fb = df00fb['StandardFantasyPoints'].mean()
avg_00hb = df00hb['StandardFantasyPoints'].mean()

avg_01qb = df01qb['StandardFantasyPoints'].mean()
avg_01wr = df01wr['StandardFantasyPoints'].mean()
avg_01te = df01te['StandardFantasyPoints'].mean()
avg_01fb = df01fb['StandardFantasyPoints'].mean()
avg_01hb = df01hb['StandardFantasyPoints'].mean()

avg_02qb = df02qb['StandardFantasyPoints'].mean()
avg_02wr = df02wr['StandardFantasyPoints'].mean()
avg_02te = df02te['StandardFantasyPoints'].mean()
avg_02fb = df02fb['StandardFantasyPoints'].mean()
avg_02hb = df02hb['StandardFantasyPoints'].mean()

avg_03qb = df03qb['StandardFantasyPoints'].mean()
avg_03wr = df03wr['StandardFantasyPoints'].mean()
avg_03te = df03te['StandardFantasyPoints'].mean()
avg_03fb = df03fb['StandardFantasyPoints'].mean()
avg_03hb = df03hb['StandardFantasyPoints'].mean()

avg_04qb = df04qb['StandardFantasyPoints'].mean()
avg_04wr = df04wr['StandardFantasyPoints'].mean()
avg_04te = df04te['StandardFantasyPoints'].mean()
avg_04fb = df04fb['StandardFantasyPoints'].mean()
avg_04hb = df04hb['StandardFantasyPoints'].mean()

avg_05qb = df05qb['StandardFantasyPoints'].mean()
avg_05wr = df05wr['StandardFantasyPoints'].mean()
avg_05te = df05te['StandardFantasyPoints'].mean()
avg_05fb = df05fb['StandardFantasyPoints'].mean()
avg_05hb = df05hb['StandardFantasyPoints'].mean()

avg_06qb = df06qb['StandardFantasyPoints'].mean()
avg_06wr = df06wr['StandardFantasyPoints'].mean()
avg_06te = df06te['StandardFantasyPoints'].mean()
avg_06fb = df06fb['StandardFantasyPoints'].mean()
avg_06hb = df06hb['StandardFantasyPoints'].mean()

avg_07qb = df07qb['StandardFantasyPoints'].mean()
avg_07wr = df07wr['StandardFantasyPoints'].mean()
avg_07te = df07te['StandardFantasyPoints'].mean()
avg_07fb = df07fb['StandardFantasyPoints'].mean()
avg_07hb = df07hb['StandardFantasyPoints'].mean()

avg_08qb = df08qb['StandardFantasyPoints'].mean()
avg_08wr = df08wr['StandardFantasyPoints'].mean()
avg_08te = df08te['StandardFantasyPoints'].mean()
avg_08fb = df08fb['StandardFantasyPoints'].mean()
avg_08hb = df08hb['StandardFantasyPoints'].mean()

avg_09qb = df09qb['StandardFantasyPoints'].mean()
avg_09wr = df09wr['StandardFantasyPoints'].mean()
avg_09te = df09te['StandardFantasyPoints'].mean()
avg_09fb = df09fb['StandardFantasyPoints'].mean()
avg_09hb = df09hb['StandardFantasyPoints'].mean()

avg_10qb = df10qb['StandardFantasyPoints'].mean()
avg_10wr = df10wr['StandardFantasyPoints'].mean()
avg_10te = df10te['StandardFantasyPoints'].mean()
avg_10fb = df10fb['StandardFantasyPoints'].mean()
avg_10hb = df10hb['StandardFantasyPoints'].mean()

avg_11qb = df11qb['StandardFantasyPoints'].mean()
avg_11wr = df11wr['StandardFantasyPoints'].mean()
avg_11te = df11te['StandardFantasyPoints'].mean()
avg_11fb = df11fb['StandardFantasyPoints'].mean()
avg_11hb = df11hb['StandardFantasyPoints'].mean()

avg_12qb = df12qb['StandardFantasyPoints'].mean()
avg_12wr = df12wr['StandardFantasyPoints'].mean()
avg_12te = df12te['StandardFantasyPoints'].mean()
avg_12fb = df12fb['StandardFantasyPoints'].mean()
avg_12hb = df12hb['StandardFantasyPoints'].mean()

avg_13qb = df13qb['StandardFantasyPoints'].mean()
avg_13wr = df13wr['StandardFantasyPoints'].mean()
avg_13te = df13te['StandardFantasyPoints'].mean()
avg_13fb = df13fb['StandardFantasyPoints'].mean()
avg_13hb = df13hb['StandardFantasyPoints'].mean()

avg_14qb = df14qb['StandardFantasyPoints'].mean()
avg_14wr = df14wr['StandardFantasyPoints'].mean()
avg_14te = df14te['StandardFantasyPoints'].mean()
avg_14fb = df14fb['StandardFantasyPoints'].mean()
avg_14hb = df14hb['StandardFantasyPoints'].mean()

avg_15qb = df15qb['StandardFantasyPoints'].mean()
avg_15wr = df15wr['StandardFantasyPoints'].mean()
avg_15te = df15te['StandardFantasyPoints'].mean()
avg_15fb = df15fb['StandardFantasyPoints'].mean()
avg_15hb = df15hb['StandardFantasyPoints'].mean()

avg_16qb = df16qb['StandardFantasyPoints'].mean()
avg_16wr = df16wr['StandardFantasyPoints'].mean()
avg_16te = df16te['StandardFantasyPoints'].mean()
avg_16fb = df16fb['StandardFantasyPoints'].mean()
avg_16hb = df16hb['StandardFantasyPoints'].mean()

avg_17qb = df17qb['StandardFantasyPoints'].mean()
avg_17wr = df17wr['StandardFantasyPoints'].mean()
avg_17te = df17te['StandardFantasyPoints'].mean()
avg_17fb = df17fb['StandardFantasyPoints'].mean()
avg_17hb = df17hb['StandardFantasyPoints'].mean()

avg_18qb = df18qb['StandardFantasyPoints'].mean()
avg_18wr = df18wr['StandardFantasyPoints'].mean()
avg_18te = df18te['StandardFantasyPoints'].mean()
avg_18fb = df18fb['StandardFantasyPoints'].mean()
avg_18hb = df18hb['StandardFantasyPoints'].mean()

avg_19qb = df19qb['StandardFantasyPoints'].mean()
avg_19wr = df19wr['StandardFantasyPoints'].mean()
avg_19te = df19te['StandardFantasyPoints'].mean()
avg_19fb = df19fb['StandardFantasyPoints'].mean()
avg_19hb = df19hb['StandardFantasyPoints'].mean()

avg_data = {'Year': [1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019],
        'QB': [avg_99qb,avg_00qb,avg_01qb,avg_02qb,avg_03qb,avg_04qb,avg_05qb,avg_06qb,avg_07qb,avg_08qb,avg_09qb,avg_10qb,avg_11qb,avg_12qb,avg_13qb,avg_14qb,avg_15qb,avg_16qb,avg_17qb,avg_18qb,avg_19qb],
        'WR': [avg_99wr,avg_00wr,avg_01wr,avg_02wr,avg_03wr,avg_04wr,avg_05wr,avg_06wr,avg_07wr,avg_08wr,avg_09wr,avg_10wr,avg_11wr,avg_12wr,avg_13wr,avg_14wr,avg_15wr,avg_16wr,avg_17wr,avg_18wr,avg_19wr],
        'TE': [avg_99te,avg_00te,avg_01te,avg_02te,avg_03te,avg_04te,avg_05te,avg_06te,avg_07te,avg_08te,avg_09te,avg_10te,avg_11te,avg_12te,avg_13te,avg_14te,avg_15te,avg_16te,avg_17te,avg_18te,avg_19te],
        'FB': [avg_99fb,avg_00fb,avg_01fb,avg_02fb,avg_03fb,avg_04fb,avg_05fb,avg_06fb,avg_07fb,avg_08fb,avg_09fb,avg_10fb,avg_11fb,avg_12fb,avg_13fb,avg_14fb,avg_15fb,avg_16fb,avg_17fb,avg_18fb,avg_19fb],
        'HB': [avg_99hb,avg_00hb,avg_01hb,avg_02hb,avg_03hb,avg_04hb,avg_05hb,avg_06hb,avg_07hb,avg_08hb,avg_09hb,avg_10hb,avg_11hb,avg_12hb,avg_13hb,avg_14hb,avg_15hb,avg_16hb,avg_17hb,avg_18hb,avg_19hb],
        }

yearly_avgdf = pd.DataFrame(data=avg_data)

#Find overall position averages
avg_qb = yearly_avgdf['QB'].mean()
avg_wr = yearly_avgdf['WR'].mean()
avg_te = yearly_avgdf['TE'].mean()
avg_fb = yearly_avgdf['FB'].mean()
avg_hb = yearly_avgdf['HB'].mean()

newrow = {'Year': 'AVG', 'QB': avg_qb, 'WR': avg_wr, 'TE': avg_te, 'FB': avg_fb, 'HB': avg_hb}

yearly_avgdf = yearly_avgdf.append(newrow, ignore_index=True)

In [ ]:
print(yearly_avgdf)

    Year         QB        WR        TE        FB        HB
0   1999  11.581724  6.143239  3.410274  3.018440  7.189030
1   2000  11.815687  5.975730  3.390294  2.651268  7.969276
2   2001  11.960479  5.929324  3.615091  2.502965  7.420424
3   2002  12.591210  6.266790  3.451225  2.464674  8.262115
4   2003  11.467695  5.831956  3.400424  2.378144  7.535279
5   2004  12.792615  6.084467  4.040081  2.013187  8.035888
6   2005  11.293450  5.915897  3.738161  2.055385  7.739312
7   2006  11.893872  5.914014  3.753378  1.777544  7.668259
8   2007  11.634835  5.982829  4.040135  1.620863  7.370277
9   2008  12.007790  5.832853  3.694271  2.081301  7.874175
10  2009  12.038932  5.903719  4.415824  1.854183  7.486067
11  2010  13.025599  5.999216  4.333805  1.633423  7.171711
12  2011  13.845554  6.061797  4.427684  2.132500  6.923894
13  2012  14.019643  6.085547  4.476031  2.054783  6.737200
14  2013  14.698553  6.102448  4.687097  1.936488  7.129194
15  2014  14.491886  5.970902  4.358096 

## Calculate EMA
The EMA, or Exponential Moving Average, is commonly used as a technical indicator in Finance however it more generally is a way of calculating averages over time so that the more recent data has a larger weight in the factoring of the average. The EMA is used in this case to acknowledge that football is a changing game and as such more recent data is more accurate to the future and present of the game then past data, but at the same time that past data can still provide inside in a limited capacity.

In [ ]:
years = len(yearly_avgdf.index) - 1
yearly_emadf = pd.Series.ewm(yearly_avgdf[['QB','WR','TE','FB','HB']], span=years).mean()
yearly_emadf = yearly_emadf.iloc[:-1]
yearly_emadf = yearly_emadf.assign(Year=[1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019])

print(yearly_emadf)

           QB        WR        TE        FB        HB  Year
0   11.581724  6.143239  3.410274  3.018440  7.189030  1999
1   11.704276  6.055496  3.399808  2.826111  7.597730  2000
2   11.797933  6.009373  3.478507  2.707982  7.532914  2001
3   12.025438  6.083198  3.470683  2.638204  7.742043  2002
4   11.891683  6.022946  3.453833  2.575837  7.692458  2003
5   12.079738  6.035787  3.576203  2.458393  7.764143  2004
6   11.932913  6.013400  3.606446  2.383138  7.759506  2005
7   11.926260  5.996464  3.631484  2.279943  7.743957  2006
8   11.880257  5.994312  3.695991  2.175904  7.684970  2007
9   11.899126  5.970424  3.695737  2.161907  7.712963  2008
10  11.918694  5.961087  3.796525  2.118836  7.681205  2009
11  12.066378  5.966175  3.868209  2.054072  7.613228  2010
12  12.294078  5.978412  3.939811  2.064109  7.525007  2011
13  12.507023  5.991633  4.005984  2.062958  7.427787  2012
14  12.768958  6.004878  4.087391  2.047842  7.392098  2013
15  12.969157  6.000930  4.118847  2.029

# Normalization of Data
## Calculate Defensive Averages
Calculate the averages of each team's defense against each position grouping every year. Using these averages compared to the EMA value of the leagues points at that position will give us our desired offset for normalization.

In [ ]:
ari_qb = []
ari_wr = []
ari_te = []
ari_fb = []
ari_hb = []
atl_qb = []
atl_wr = []
atl_te = []
atl_fb = []
atl_hb = []
car_qb = []
car_wr = []
car_te = []
car_fb = []
car_hb = []
chi_qb = []
chi_wr = []
chi_te = []
chi_fb = []
chi_hb = []
dal_qb = []
dal_wr = []
dal_te = []
dal_fb = []
dal_hb = []
det_qb = []
det_wr = []
det_te = []
det_fb = []
det_hb = []
gnb_qb = []
gnb_wr = []
gnb_te = []
gnb_fb = []
gnb_hb = []
stl_qb = []
stl_wr = []
stl_te = []
stl_fb = []
stl_hb = []
min_qb = []
min_wr = []
min_te = []
min_fb = []
min_hb = []
nor_qb = []
nor_wr = []
nor_te = []
nor_fb = []
nor_hb = []
nyg_qb = []
nyg_wr = []
nyg_te = []
nyg_fb = []
nyg_hb = []
phi_qb = []
phi_wr = []
phi_te = []
phi_fb = []
phi_hb = []
sfo_qb = []
sfo_wr = []
sfo_te = []
sfo_fb = []
sfo_hb = []
sea_qb = []
sea_wr = []
sea_te = []
sea_fb = []
sea_hb = []
tam_qb = []
tam_wr = []
tam_te = []
tam_fb = []
tam_hb = []
was_qb = []
was_wr = []
was_te = []
was_fb = []
was_hb = []

bal_qb = []
bal_wr = []
bal_te = []
bal_fb = []
bal_hb = []
buf_qb = []
buf_wr = []
buf_te = []
buf_fb = []
buf_hb = []
cin_qb = []
cin_wr = []
cin_te = []
cin_fb = []
cin_hb = []
cle_qb = []
cle_wr = []
cle_te = []
cle_fb = []
cle_hb = []
den_qb = []
den_wr = []
den_te = []
den_fb = []
den_hb = []
hou_qb = []
hou_wr = []
hou_te = []
hou_fb = []
hou_hb = []
ind_qb = []
ind_wr = []
ind_te = []
ind_fb = []
ind_hb = []
jax_qb = []
jax_wr = []
jax_te = []
jax_fb = []
jax_hb = []
kan_qb = []
kan_wr = []
kan_te = []
kan_fb = []
kan_hb = []
oak_qb = []
oak_wr = []
oak_te = []
oak_fb = []
oak_hb = []
sdg_qb = []
sdg_wr = []
sdg_te = []
sdg_fb = []
sdg_hb = []
mia_qb = []
mia_wr = []
mia_te = []
mia_fb = []
mia_hb = []
nwe_qb = []
nwe_wr = []
nwe_te = []
nwe_fb = []
nwe_hb = []
nyj_qb = []
nyj_wr = []
nyj_te = []
nyj_fb = []
nyj_hb = []
pit_qb = []
pit_wr = []
pit_te = []
pit_fb = []
pit_hb = []
ten_qb = []
ten_wr = []
ten_te = []
ten_fb = []
ten_hb = []

for i in range(1999, 2020):
  tempdf = completeDF[completeDF.Year == str(i) ]
  temp2df_qb = tempdf[tempdf.Pos == 'QB']
  temp2df_wr = tempdf[tempdf.Pos == 'WR']
  temp2df_te = tempdf[tempdf.Pos == 'TE']
  temp2df_fb = tempdf[tempdf.Pos == 'FB']
  temp2df_hb = tempdf[tempdf.Pos == 'HB']

  ari_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'ARI', 'StandardFantasyPoints'].mean())
  ari_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'ARI', 'StandardFantasyPoints'].mean())
  ari_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'ARI', 'StandardFantasyPoints'].mean())
  ari_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'ARI', 'StandardFantasyPoints'].mean())
  ari_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'ARI', 'StandardFantasyPoints'].mean())

  atl_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'ATL', 'StandardFantasyPoints'].mean())
  atl_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'ATL', 'StandardFantasyPoints'].mean())
  atl_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'ATL', 'StandardFantasyPoints'].mean())
  atl_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'ATL', 'StandardFantasyPoints'].mean())
  atl_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'ATL', 'StandardFantasyPoints'].mean())

  car_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'CAR', 'StandardFantasyPoints'].mean())
  car_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'CAR', 'StandardFantasyPoints'].mean())
  car_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'CAR', 'StandardFantasyPoints'].mean())
  car_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'CAR', 'StandardFantasyPoints'].mean())
  car_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'CAR', 'StandardFantasyPoints'].mean())

  chi_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'CHI', 'StandardFantasyPoints'].mean())
  chi_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'CHI', 'StandardFantasyPoints'].mean())
  chi_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'CHI', 'StandardFantasyPoints'].mean())
  chi_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'CHI', 'StandardFantasyPoints'].mean())
  chi_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'CHI', 'StandardFantasyPoints'].mean())

  dal_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'DAL', 'StandardFantasyPoints'].mean())
  dal_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'DAL', 'StandardFantasyPoints'].mean())
  dal_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'DAL', 'StandardFantasyPoints'].mean())
  dal_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'DAL', 'StandardFantasyPoints'].mean())
  dal_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'DAL', 'StandardFantasyPoints'].mean())

  det_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'DET', 'StandardFantasyPoints'].mean())
  det_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'DET', 'StandardFantasyPoints'].mean())
  det_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'DET', 'StandardFantasyPoints'].mean())
  det_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'DET', 'StandardFantasyPoints'].mean())
  det_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'DET', 'StandardFantasyPoints'].mean())

  gnb_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'GNB', 'StandardFantasyPoints'].mean())
  gnb_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'GNB', 'StandardFantasyPoints'].mean())
  gnb_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'GNB', 'StandardFantasyPoints'].mean())
  gnb_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'GNB', 'StandardFantasyPoints'].mean())
  gnb_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'GNB', 'StandardFantasyPoints'].mean())

  stl_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'STL', 'StandardFantasyPoints'].mean())
  stl_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'STL', 'StandardFantasyPoints'].mean())
  stl_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'STL', 'StandardFantasyPoints'].mean())
  stl_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'STL', 'StandardFantasyPoints'].mean())
  stl_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'STL', 'StandardFantasyPoints'].mean())

  min_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'MIN', 'StandardFantasyPoints'].mean())
  min_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'MIN', 'StandardFantasyPoints'].mean())
  min_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'MIN', 'StandardFantasyPoints'].mean())
  min_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'MIN', 'StandardFantasyPoints'].mean())
  min_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'MIN', 'StandardFantasyPoints'].mean())

  nor_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'NOR', 'StandardFantasyPoints'].mean())
  nor_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'NOR', 'StandardFantasyPoints'].mean())
  nor_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'NOR', 'StandardFantasyPoints'].mean())
  nor_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'NOR', 'StandardFantasyPoints'].mean())
  nor_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'NOR', 'StandardFantasyPoints'].mean())

  nyg_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'NYG', 'StandardFantasyPoints'].mean())
  nyg_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'NYG', 'StandardFantasyPoints'].mean())
  nyg_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'NYG', 'StandardFantasyPoints'].mean())
  nyg_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'NYG', 'StandardFantasyPoints'].mean())
  nyg_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'NYG', 'StandardFantasyPoints'].mean())

  phi_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'PHI', 'StandardFantasyPoints'].mean())
  phi_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'PHI', 'StandardFantasyPoints'].mean())
  phi_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'PHI', 'StandardFantasyPoints'].mean())
  phi_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'PHI', 'StandardFantasyPoints'].mean())
  phi_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'PHI', 'StandardFantasyPoints'].mean())

  sfo_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'SFO', 'StandardFantasyPoints'].mean())
  sfo_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'SFO', 'StandardFantasyPoints'].mean())
  sfo_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'SFO', 'StandardFantasyPoints'].mean())
  sfo_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'SFO', 'StandardFantasyPoints'].mean())
  sfo_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'SFO', 'StandardFantasyPoints'].mean())

  sea_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'SEA', 'StandardFantasyPoints'].mean())
  sea_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'SEA', 'StandardFantasyPoints'].mean())
  sea_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'SEA', 'StandardFantasyPoints'].mean())
  sea_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'SEA', 'StandardFantasyPoints'].mean())
  sea_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'SEA', 'StandardFantasyPoints'].mean())

  tam_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'TAM', 'StandardFantasyPoints'].mean())
  tam_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'TAM', 'StandardFantasyPoints'].mean())
  tam_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'TAM', 'StandardFantasyPoints'].mean())
  tam_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'TAM', 'StandardFantasyPoints'].mean())
  tam_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'TAM', 'StandardFantasyPoints'].mean())

  was_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'WAS', 'StandardFantasyPoints'].mean())
  was_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'WAS', 'StandardFantasyPoints'].mean())
  was_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'WAS', 'StandardFantasyPoints'].mean())
  was_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'WAS', 'StandardFantasyPoints'].mean())
  was_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'WAS', 'StandardFantasyPoints'].mean())

  bal_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'BAL', 'StandardFantasyPoints'].mean())
  bal_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'BAL', 'StandardFantasyPoints'].mean())
  bal_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'BAL', 'StandardFantasyPoints'].mean())
  bal_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'BAL', 'StandardFantasyPoints'].mean())
  bal_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'BAL', 'StandardFantasyPoints'].mean())

  buf_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'BUF', 'StandardFantasyPoints'].mean())
  buf_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'BUF', 'StandardFantasyPoints'].mean())
  buf_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'BUF', 'StandardFantasyPoints'].mean())
  buf_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'BUF', 'StandardFantasyPoints'].mean())
  buf_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'BUF', 'StandardFantasyPoints'].mean())

  cin_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'CIN', 'StandardFantasyPoints'].mean())
  cin_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'CIN', 'StandardFantasyPoints'].mean())
  cin_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'CIN', 'StandardFantasyPoints'].mean())
  cin_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'CIN', 'StandardFantasyPoints'].mean())
  cin_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'CIN', 'StandardFantasyPoints'].mean())

  cle_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'CLE', 'StandardFantasyPoints'].mean())
  cle_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'CLE', 'StandardFantasyPoints'].mean())
  cle_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'CLE', 'StandardFantasyPoints'].mean())
  cle_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'CLE', 'StandardFantasyPoints'].mean())
  cle_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'CLE', 'StandardFantasyPoints'].mean())

  den_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'DEN', 'StandardFantasyPoints'].mean())
  den_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'DEN', 'StandardFantasyPoints'].mean())
  den_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'DEN', 'StandardFantasyPoints'].mean())
  den_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'DEN', 'StandardFantasyPoints'].mean())
  den_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'DEN', 'StandardFantasyPoints'].mean())

  hou_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'HOU', 'StandardFantasyPoints'].mean())
  hou_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'HOU', 'StandardFantasyPoints'].mean())
  hou_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'HOU', 'StandardFantasyPoints'].mean())
  hou_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'HOU', 'StandardFantasyPoints'].mean())
  hou_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'HOU', 'StandardFantasyPoints'].mean())

  ind_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'IND', 'StandardFantasyPoints'].mean())
  ind_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'IND', 'StandardFantasyPoints'].mean())
  ind_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'IND', 'StandardFantasyPoints'].mean())
  ind_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'IND', 'StandardFantasyPoints'].mean())
  ind_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'IND', 'StandardFantasyPoints'].mean())

  jax_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'JAX', 'StandardFantasyPoints'].mean())
  jax_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'JAX', 'StandardFantasyPoints'].mean())
  jax_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'JAX', 'StandardFantasyPoints'].mean())
  jax_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'JAX', 'StandardFantasyPoints'].mean())
  jax_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'JAX', 'StandardFantasyPoints'].mean())

  kan_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'KAN', 'StandardFantasyPoints'].mean())
  kan_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'KAN', 'StandardFantasyPoints'].mean())
  kan_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'KAN', 'StandardFantasyPoints'].mean())
  kan_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'KAN', 'StandardFantasyPoints'].mean())
  kan_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'KAN', 'StandardFantasyPoints'].mean())

  oak_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'OAK', 'StandardFantasyPoints'].mean())
  oak_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'OAK', 'StandardFantasyPoints'].mean())
  oak_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'OAK', 'StandardFantasyPoints'].mean())
  oak_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'OAK', 'StandardFantasyPoints'].mean())
  oak_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'OAK', 'StandardFantasyPoints'].mean())

  sdg_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'SDG', 'StandardFantasyPoints'].mean())
  sdg_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'SDG', 'StandardFantasyPoints'].mean())
  sdg_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'SDG', 'StandardFantasyPoints'].mean())
  sdg_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'SDG', 'StandardFantasyPoints'].mean())
  sdg_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'SDG', 'StandardFantasyPoints'].mean())

  mia_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'MIA', 'StandardFantasyPoints'].mean())
  mia_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'MIA', 'StandardFantasyPoints'].mean())
  mia_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'MIA', 'StandardFantasyPoints'].mean())
  mia_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'MIA', 'StandardFantasyPoints'].mean())
  mia_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'MIA', 'StandardFantasyPoints'].mean())

  nwe_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'NWE', 'StandardFantasyPoints'].mean())
  nwe_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'NWE', 'StandardFantasyPoints'].mean())
  nwe_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'NWE', 'StandardFantasyPoints'].mean())
  nwe_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'NWE', 'StandardFantasyPoints'].mean())
  nwe_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'NWE', 'StandardFantasyPoints'].mean())

  nyj_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'NYJ', 'StandardFantasyPoints'].mean())
  nyj_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'NYJ', 'StandardFantasyPoints'].mean())
  nyj_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'NYJ', 'StandardFantasyPoints'].mean())
  nyj_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'NYJ', 'StandardFantasyPoints'].mean())
  nyj_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'NYJ', 'StandardFantasyPoints'].mean())

  pit_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'PIT', 'StandardFantasyPoints'].mean())
  pit_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'PIT', 'StandardFantasyPoints'].mean())
  pit_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'PIT', 'StandardFantasyPoints'].mean())
  pit_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'PIT', 'StandardFantasyPoints'].mean())
  pit_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'PIT', 'StandardFantasyPoints'].mean())

  ten_qb.append(temp2df_qb.loc[temp2df_qb['Opp'] == 'TEN', 'StandardFantasyPoints'].mean())
  ten_wr.append(temp2df_wr.loc[temp2df_wr['Opp'] == 'TEN', 'StandardFantasyPoints'].mean())
  ten_te.append(temp2df_te.loc[temp2df_te['Opp'] == 'TEN', 'StandardFantasyPoints'].mean())
  ten_fb.append(temp2df_fb.loc[temp2df_fb['Opp'] == 'TEN', 'StandardFantasyPoints'].mean())
  ten_hb.append(temp2df_hb.loc[temp2df_hb['Opp'] == 'TEN', 'StandardFantasyPoints'].mean())

## Calculate Normalization Offsets

In [ ]:
qb_data = {
    'Year': [1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019],
    'ARI': ari_qb,
    'ATL': atl_qb,
    'CAR': car_qb,
    'CHI': chi_qb,
    'DAL': dal_qb,
    'DET': det_qb,
    'GNB': gnb_qb,
    'STL': stl_qb,
    'MIN': min_qb,
    'NOR': nor_qb,
    'NYG': nyg_qb,
    'PHI': phi_qb,
    'SFO': sfo_qb,
    'SEA': sea_qb,
    'TAM': tam_qb,
    'WAS': was_qb,
    'BAL': bal_qb,
    'BUF': buf_qb,
    'CIN': cin_qb,
    'CLE': cle_qb,
    'DEN': den_qb,
    'HOU': hou_qb,
    'IND': ind_qb,
    'JAX': jax_qb,
    'KAN': kan_qb,
    'OAK': oak_qb,
    'SDG': sdg_qb,
    'MIA': mia_qb,
    'NWE': nwe_qb,
    'NYJ': nyj_qb,
    'PIT': pit_qb,
    'TEN': ten_qb
}
qb_offsetdf = pd.DataFrame(data=qb_data)

wr_data = {
    'Year': [1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019],
    'ARI': ari_wr,
    'ATL': atl_wr,
    'CAR': car_wr,
    'CHI': chi_wr,
    'DAL': dal_wr,
    'DET': det_wr,
    'GNB': gnb_wr,
    'STL': stl_wr,
    'MIN': min_wr,
    'NOR': nor_wr,
    'NYG': nyg_wr,
    'PHI': phi_wr,
    'SFO': sfo_wr,
    'SEA': sea_wr,
    'TAM': tam_wr,
    'WAS': was_wr,
    'BAL': bal_wr,
    'BUF': buf_wr,
    'CIN': cin_wr,
    'CLE': cle_wr,
    'DEN': den_wr,
    'HOU': hou_wr,
    'IND': ind_wr,
    'JAX': jax_wr,
    'KAN': kan_wr,
    'OAK': oak_wr,
    'SDG': sdg_wr,
    'MIA': mia_wr,
    'NWE': nwe_wr,
    'NYJ': nyj_wr,
    'PIT': pit_wr,
    'TEN': ten_wr
}
wr_offsetdf = pd.DataFrame(data=wr_data)

te_data = {
    'Year': [1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019],
    'ARI': ari_te,
    'ATL': atl_te,
    'CAR': car_te,
    'CHI': chi_te,
    'DAL': dal_te,
    'DET': det_te,
    'GNB': gnb_te,
    'STL': stl_te,
    'MIN': min_te,
    'NOR': nor_te,
    'NYG': nyg_te,
    'PHI': phi_te,
    'SFO': sfo_te,
    'SEA': sea_te,
    'TAM': tam_te,
    'WAS': was_te,
    'BAL': bal_te,
    'BUF': buf_te,
    'CIN': cin_te,
    'CLE': cle_te,
    'DEN': den_te,
    'HOU': hou_te,
    'IND': ind_te,
    'JAX': jax_te,
    'KAN': kan_te,
    'OAK': oak_te,
    'SDG': sdg_te,
    'MIA': mia_te,
    'NWE': nwe_te,
    'NYJ': nyj_te,
    'PIT': pit_te,
    'TEN': ten_te
}
te_offsetdf = pd.DataFrame(data=te_data)

fb_data = {
    'Year': [1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019],
    'ARI': ari_fb,
    'ATL': atl_fb,
    'CAR': car_fb,
    'CHI': chi_fb,
    'DAL': dal_fb,
    'DET': det_fb,
    'GNB': gnb_fb,
    'STL': stl_fb,
    'MIN': min_fb,
    'NOR': nor_fb,
    'NYG': nyg_fb,
    'PHI': phi_fb,
    'SFO': sfo_fb,
    'SEA': sea_fb,
    'TAM': tam_fb,
    'WAS': was_fb,
    'BAL': bal_fb,
    'BUF': buf_fb,
    'CIN': cin_fb,
    'CLE': cle_fb,
    'DEN': den_fb,
    'HOU': hou_fb,
    'IND': ind_fb,
    'JAX': jax_fb,
    'KAN': kan_fb,
    'OAK': oak_fb,
    'SDG': sdg_fb,
    'MIA': mia_fb,
    'NWE': nwe_fb,
    'NYJ': nyj_fb,
    'PIT': pit_fb,
    'TEN': ten_fb
}
fb_offsetdf = pd.DataFrame(data=fb_data)

hb_data = {
    'Year': [1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019],
    'ARI': ari_hb,
    'ATL': atl_hb,
    'CAR': car_hb,
    'CHI': chi_hb,
    'DAL': dal_hb,
    'DET': det_hb,
    'GNB': gnb_hb,
    'STL': stl_hb,
    'MIN': min_hb,
    'NOR': nor_hb,
    'NYG': nyg_hb,
    'PHI': phi_hb,
    'SFO': sfo_hb,
    'SEA': sea_hb,
    'TAM': tam_hb,
    'WAS': was_hb,
    'BAL': bal_hb,
    'BUF': buf_hb,
    'CIN': cin_hb,
    'CLE': cle_hb,
    'DEN': den_hb,
    'HOU': hou_hb,
    'IND': ind_hb,
    'JAX': jax_hb,
    'KAN': kan_hb,
    'OAK': oak_hb,
    'SDG': sdg_hb,
    'MIA': mia_hb,
    'NWE': nwe_hb,
    'NYJ': nyj_hb,
    'PIT': pit_hb,
    'TEN': ten_hb
}
hb_offsetdf = pd.DataFrame(data=hb_data)

qbtempdf = yearly_emadf['QB']
wrtempdf = yearly_emadf['WR']
tetempdf = yearly_emadf['TE']
fbtempdf = yearly_emadf['FB']
hbtempdf = yearly_emadf['HB']

qb_offsetdf = qbtempdf - qb_offsetdf.transpose()
wr_offsetdf = wrtempdf - wr_offsetdf.transpose()
te_offsetdf = tetempdf - te_offsetdf.transpose()
fb_offsetdf = fbtempdf - fb_offsetdf.transpose()
hb_offsetdf = hbtempdf - hb_offsetdf.transpose()

qb_offsetdf = qb_offsetdf.transpose()
wr_offsetdf = wr_offsetdf.transpose()
te_offsetdf = te_offsetdf.transpose()
fb_offsetdf = fb_offsetdf.transpose()
hb_offsetdf = hb_offsetdf.transpose()

qb_offsetdf = qb_offsetdf.drop('Year', axis=1)
wr_offsetdf = wr_offsetdf.drop('Year', axis=1)
te_offsetdf = te_offsetdf.drop('Year', axis=1)
fb_offsetdf = fb_offsetdf.drop('Year', axis=1)
hb_offsetdf = hb_offsetdf.drop('Year', axis=1)

qb_offsetdf = qb_offsetdf.assign(Year=[1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019])
wr_offsetdf = wr_offsetdf.assign(Year=[1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019])
te_offsetdf = te_offsetdf.assign(Year=[1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019])
fb_offsetdf = fb_offsetdf.assign(Year=[1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019])
hb_offsetdf = hb_offsetdf.assign(Year=[1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019])

print(qb_offsetdf)

         ARI       ATL       CAR       CHI  ...       NYJ       PIT       TEN  Year
0  -1.455923 -0.284276 -2.418276 -1.920381  ... -0.237099 -1.011217 -0.532562  1999
1  -1.053502 -3.026974 -1.614474 -2.532867  ...  1.364276  3.195855  3.403223  2000
2  -1.537622 -4.973317 -1.483178  0.747345  ...  3.866933  0.610874 -3.902067  2001
3  -1.585673  0.236550  1.549649 -2.295673  ...  1.783333 -0.007503 -0.067895  2002
4  -2.305317 -0.808317 -0.260540  1.459460  ...  0.521683 -0.344567  0.075212  2003
5   2.713422 -0.641215  0.472238 -0.338040  ... -0.436453  3.072238 -3.542762  2004
6   2.892913  2.543913  2.831736  5.206246  ...  2.671736  0.607913 -1.908199  2005
7  -2.258740  1.133628  0.670470  2.389789  ...  1.026260  0.740704  0.740546  2006
8  -0.917521 -1.772684 -0.539743  1.361368  ...  0.605257  1.600257  1.615813  2007
9  -3.410874 -2.853374 -0.585580  1.487126  ... -2.775874  3.357792  4.188649  2008
10 -3.181306 -2.300195  2.053694 -0.241306  ...  3.656194 -0.277973 -2.39183

## Normalize Individual Performances

In [ ]:
def adjustpts(df, df_offset):
  temp_adjpts = []
  df = df[['Player','Pos','Tm','Week','Opp','StandardFantasyPoints']]
  for i in range(len(df)):
    df2 = df.iloc[i]
    opponent = df2['Opp']
    df3 = df_offset[opponent]
    temp_adjpts.append(df2['StandardFantasyPoints'] + df3[0])
  df['AdjPts'] = temp_adjpts
  return df

df99qb = adjustpts(df99qb, qb_offsetdf)
df99wr = adjustpts(df99wr, wr_offsetdf)
df99te = adjustpts(df99te, te_offsetdf)
df99fb = adjustpts(df99fb, fb_offsetdf)
df99hb = adjustpts(df99hb, hb_offsetdf)
df00qb = adjustpts(df00qb, qb_offsetdf)
df00wr = adjustpts(df00wr, wr_offsetdf)
df00te = adjustpts(df00te, te_offsetdf)
df00fb = adjustpts(df00fb, fb_offsetdf)
df00hb = adjustpts(df00hb, hb_offsetdf)
df01qb = adjustpts(df01qb, qb_offsetdf)
df01wr = adjustpts(df01wr, wr_offsetdf)
df01te = adjustpts(df01te, te_offsetdf)
df01fb = adjustpts(df01fb, fb_offsetdf)
df01hb = adjustpts(df01hb, hb_offsetdf)
df02qb = adjustpts(df02qb, qb_offsetdf)
df02wr = adjustpts(df02wr, wr_offsetdf)
df02te = adjustpts(df02te, te_offsetdf)
df02fb = adjustpts(df02fb, fb_offsetdf)
df02hb = adjustpts(df02hb, hb_offsetdf)
df03qb = adjustpts(df03qb, qb_offsetdf)
df03wr = adjustpts(df03wr, wr_offsetdf)
df03te = adjustpts(df03te, te_offsetdf)
df03fb = adjustpts(df03fb, fb_offsetdf)
df03hb = adjustpts(df03hb, hb_offsetdf)
df04qb = adjustpts(df04qb, qb_offsetdf)
df04wr = adjustpts(df04wr, wr_offsetdf)
df04te = adjustpts(df04te, te_offsetdf)
df04fb = adjustpts(df04fb, fb_offsetdf)
df04hb = adjustpts(df04hb, hb_offsetdf)
df05qb = adjustpts(df05qb, qb_offsetdf)
df05wr = adjustpts(df05wr, wr_offsetdf)
df05te = adjustpts(df05te, te_offsetdf)
df05fb = adjustpts(df05fb, fb_offsetdf)
df05hb = adjustpts(df05hb, hb_offsetdf)
df06qb = adjustpts(df06qb, qb_offsetdf)
df06wr = adjustpts(df06wr, wr_offsetdf)
df06te = adjustpts(df06te, te_offsetdf)
df06fb = adjustpts(df06fb, fb_offsetdf)
df06hb = adjustpts(df06hb, hb_offsetdf)
df07qb = adjustpts(df07qb, qb_offsetdf)
df07wr = adjustpts(df07wr, wr_offsetdf)
df07te = adjustpts(df07te, te_offsetdf)
df07fb = adjustpts(df07fb, fb_offsetdf)
df07hb = adjustpts(df07hb, hb_offsetdf)
df08qb = adjustpts(df08qb, qb_offsetdf)
df08wr = adjustpts(df08wr, wr_offsetdf)
df08te = adjustpts(df08te, te_offsetdf)
df08fb = adjustpts(df08fb, fb_offsetdf)
df08hb = adjustpts(df08hb, hb_offsetdf)
df09qb = adjustpts(df09qb, qb_offsetdf)
df09wr = adjustpts(df09wr, wr_offsetdf)
df09te = adjustpts(df09te, te_offsetdf)
df09fb = adjustpts(df09fb, fb_offsetdf)
df09hb = adjustpts(df09hb, hb_offsetdf)
df10qb = adjustpts(df10qb, qb_offsetdf)
df10wr = adjustpts(df10wr, wr_offsetdf)
df10te = adjustpts(df10te, te_offsetdf)
df10fb = adjustpts(df10fb, fb_offsetdf)
df10hb = adjustpts(df10hb, hb_offsetdf)
df11qb = adjustpts(df11qb, qb_offsetdf)
df11wr = adjustpts(df11wr, wr_offsetdf)
df11te = adjustpts(df11te, te_offsetdf)
df11fb = adjustpts(df11fb, fb_offsetdf)
df11hb = adjustpts(df11hb, hb_offsetdf)
df12qb = adjustpts(df12qb, qb_offsetdf)
df12wr = adjustpts(df12wr, wr_offsetdf)
df12te = adjustpts(df12te, te_offsetdf)
df12fb = adjustpts(df12fb, fb_offsetdf)
df12hb = adjustpts(df12hb, hb_offsetdf)
df13qb = adjustpts(df13qb, qb_offsetdf)
df13wr = adjustpts(df13wr, wr_offsetdf)
df13te = adjustpts(df13te, te_offsetdf)
df13fb = adjustpts(df13fb, fb_offsetdf)
df13hb = adjustpts(df13hb, hb_offsetdf)
df14qb = adjustpts(df14qb, qb_offsetdf)
df14wr = adjustpts(df14wr, wr_offsetdf)
df14te = adjustpts(df14te, te_offsetdf)
df14fb = adjustpts(df14fb, fb_offsetdf)
df14hb = adjustpts(df14hb, hb_offsetdf)
df15qb = adjustpts(df15qb, qb_offsetdf)
df15wr = adjustpts(df15wr, wr_offsetdf)
df15te = adjustpts(df15te, te_offsetdf)
df15fb = adjustpts(df15fb, fb_offsetdf)
df15hb = adjustpts(df15hb, hb_offsetdf)
df16qb = adjustpts(df16qb, qb_offsetdf)
df16wr = adjustpts(df16wr, wr_offsetdf)
df16te = adjustpts(df16te, te_offsetdf)
df16fb = adjustpts(df16fb, fb_offsetdf)
df16hb = adjustpts(df16hb, hb_offsetdf)
df17qb = adjustpts(df17qb, qb_offsetdf)
df17wr = adjustpts(df17wr, wr_offsetdf)
df17te = adjustpts(df17te, te_offsetdf)
df17fb = adjustpts(df17fb, fb_offsetdf)
df17hb = adjustpts(df17hb, hb_offsetdf)
df18qb = adjustpts(df18qb, qb_offsetdf)
df18wr = adjustpts(df18wr, wr_offsetdf)
df18te = adjustpts(df18te, te_offsetdf)
df18fb = adjustpts(df18fb, fb_offsetdf)
df18hb = adjustpts(df18hb, hb_offsetdf)
df19qb = adjustpts(df19qb, qb_offsetdf)
df19wr = adjustpts(df19wr, wr_offsetdf)
df19te = adjustpts(df19te, te_offsetdf)
df19fb = adjustpts(df19fb, fb_offsetdf)
df19hb = adjustpts(df19hb, hb_offsetdf)

In [ ]:
print(df19qb)

                Player Pos   Tm Week  Opp  StandardFantasyPoints     AdjPts
88575    Lamar Jackson  QB  BAL    1  MIA                  33.56  35.000548
88576     Dak Prescott  QB  DAL    1  NYG                  33.40  34.121724
88577   Robert Griffin  QB  BAL    1  MIA                   7.10   8.540548
88578  Patrick Mahomes  QB  KAN    1  JAX                  27.32  30.033542
88579     Kirk Cousins  QB  MIN    1  ATL                  14.32  14.035724
...                ...  ..  ...  ...  ...                    ...        ...
92865    Devlin Hodges  QB  PIT   17  BAL                   1.70   3.144724
92866   Robert Griffin  QB  BAL   17  PIT                   6.84   5.828783
92867     David Blough  QB  DET   17  GNB                  11.88  12.289502
92952      Taysom Hill  QB  NOR   17  CAR                  10.90   8.481724
92963   Trace McSorley  QB  BAL   17  PIT                   0.10  -0.911217

[552 rows x 7 columns]


# Calculate Yearly Position Leaders
## Calculate Player's Season Totals
Add up all the performances for each player across the entire season to get the total AdjPts earned for each season per player. Then sort each yearly position list by total AdjPts to find the leader.

In [ ]:
final_99qb = df99qb[['Player','Pos','Tm','AdjPts']]
final_99qb = final_99qb.groupby(['Player']).sum()
final_99qb = final_99qb.sort_values('AdjPts')
qb99_top = final_99qb.iloc[-1]
qb99_top = qb99_top.name
final_99wr = df99wr[['Player','Pos','Tm','AdjPts']]
final_99wr = final_99wr.groupby(['Player']).sum()
final_99wr = final_99wr.sort_values('AdjPts')
wr99_top = final_99wr.iloc[-1]
wr99_top = wr99_top.name
final_99te = df99te[['Player','Pos','Tm','AdjPts']]
final_99te = final_99te.groupby(['Player']).sum()
final_99te = final_99te.sort_values('AdjPts')
te99_top = final_99te.iloc[-1]
te99_top = te99_top.name
final_99fb = df99fb[['Player','Pos','Tm','AdjPts']]
final_99fb = final_99fb.groupby(['Player']).sum()
final_99fb = final_99fb.sort_values('AdjPts')
fb99_top = final_99fb.iloc[-1]
fb99_top = fb99_top.name
final_99hb = df99hb[['Player','Pos','Tm','AdjPts']]
final_99hb = final_99hb.groupby(['Player']).sum()
final_99hb = final_99hb.sort_values('AdjPts')
hb99_top = final_99hb.iloc[-1]
hb99_top = hb99_top.name
final_00qb = df00qb[['Player','Pos','Tm','AdjPts']]
final_00qb = final_00qb.groupby(['Player']).sum()
final_00qb = final_00qb.sort_values('AdjPts')
qb00_top = final_00qb.iloc[-1]
qb00_top = qb00_top.name
final_00wr = df00wr[['Player','Pos','Tm','AdjPts']]
final_00wr = final_00wr.groupby(['Player']).sum()
final_00wr = final_00wr.sort_values('AdjPts')
wr00_top = final_00wr.iloc[-1]
wr00_top = wr00_top.name
final_00te = df00te[['Player','Pos','Tm','AdjPts']]
final_00te = final_00te.groupby(['Player']).sum()
final_00te = final_00te.sort_values('AdjPts')
te00_top = final_00te.iloc[-1]
te00_top = te00_top.name
final_00fb = df00fb[['Player','Pos','Tm','AdjPts']]
final_00fb = final_00fb.groupby(['Player']).sum()
final_00fb = final_00fb.sort_values('AdjPts')
fb00_top = final_00fb.iloc[-1]
fb00_top = fb00_top.name
final_00hb = df00hb[['Player','Pos','Tm','AdjPts']]
final_00hb = final_00hb.groupby(['Player']).sum()
final_00hb = final_00hb.sort_values('AdjPts')
hb00_top = final_00hb.iloc[-1]
hb00_top = hb00_top.name
final_01qb = df01qb[['Player','Pos','Tm','AdjPts']]
final_01qb = final_01qb.groupby(['Player']).sum()
final_01qb = final_01qb.sort_values('AdjPts')
qb01_top = final_01qb.iloc[-1]
qb01_top = qb01_top.name
final_01wr = df01wr[['Player','Pos','Tm','AdjPts']]
final_01wr = final_01wr.groupby(['Player']).sum()
final_01wr = final_01wr.sort_values('AdjPts')
wr01_top = final_01wr.iloc[-1]
wr01_top = wr01_top.name
final_01te = df01te[['Player','Pos','Tm','AdjPts']]
final_01te = final_01te.groupby(['Player']).sum()
final_01te = final_01te.sort_values('AdjPts')
te01_top = final_01te.iloc[-1]
te01_top = te01_top.name
final_01fb = df01fb[['Player','Pos','Tm','AdjPts']]
final_01fb = final_01fb.groupby(['Player']).sum()
final_01fb = final_01fb.sort_values('AdjPts')
fb01_top = final_01fb.iloc[-1]
fb01_top = fb01_top.name
final_01hb = df01hb[['Player','Pos','Tm','AdjPts']]
final_01hb = final_01hb.groupby(['Player']).sum()
final_01hb = final_01hb.sort_values('AdjPts')
hb01_top = final_01hb.iloc[-1]
hb01_top = hb01_top.name
final_02qb = df02qb[['Player','Pos','Tm','AdjPts']]
final_02qb = final_02qb.groupby(['Player']).sum()
final_02qb = final_02qb.sort_values('AdjPts')
qb02_top = final_02qb.iloc[-1]
qb02_top = qb02_top.name
final_02wr = df02wr[['Player','Pos','Tm','AdjPts']]
final_02wr = final_02wr.groupby(['Player']).sum()
final_02wr = final_02wr.sort_values('AdjPts')
wr02_top = final_02wr.iloc[-1]
wr02_top = wr02_top.name
final_02te = df02te[['Player','Pos','Tm','AdjPts']]
final_02te = final_02te.groupby(['Player']).sum()
final_02te = final_02te.sort_values('AdjPts')
te02_top = final_02te.iloc[-1]
te02_top = te02_top.name
final_02fb = df02fb[['Player','Pos','Tm','AdjPts']]
final_02fb = final_02fb.groupby(['Player']).sum()
final_02fb = final_02fb.sort_values('AdjPts')
fb02_top = final_02fb.iloc[-1]
fb02_top = fb02_top.name
final_02hb = df02hb[['Player','Pos','Tm','AdjPts']]
final_02hb = final_02hb.groupby(['Player']).sum()
final_02hb = final_02hb.sort_values('AdjPts')
hb02_top = final_02hb.iloc[-1]
hb02_top = hb02_top.name
final_03qb = df03qb[['Player','Pos','Tm','AdjPts']]
final_03qb = final_03qb.groupby(['Player']).sum()
final_03qb = final_03qb.sort_values('AdjPts')
qb03_top = final_03qb.iloc[-1]
qb03_top = qb03_top.name
final_03wr = df03wr[['Player','Pos','Tm','AdjPts']]
final_03wr = final_03wr.groupby(['Player']).sum()
final_03wr = final_03wr.sort_values('AdjPts')
wr03_top = final_03wr.iloc[-1]
wr03_top = wr03_top.name
final_03te = df03te[['Player','Pos','Tm','AdjPts']]
final_03te = final_03te.groupby(['Player']).sum()
final_03te = final_03te.sort_values('AdjPts')
te03_top = final_03te.iloc[-1]
te03_top = te03_top.name
final_03fb = df03fb[['Player','Pos','Tm','AdjPts']]
final_03fb = final_03fb.groupby(['Player']).sum()
final_03fb = final_03fb.sort_values('AdjPts')
fb03_top = final_03fb.iloc[-1]
fb03_top = fb03_top.name
final_03hb = df03hb[['Player','Pos','Tm','AdjPts']]
final_03hb = final_03hb.groupby(['Player']).sum()
final_03hb = final_03hb.sort_values('AdjPts')
hb03_top = final_03hb.iloc[-1]
hb03_top = hb03_top.name
final_04qb = df04qb[['Player','Pos','Tm','AdjPts']]
final_04qb = final_04qb.groupby(['Player']).sum()
final_04qb = final_04qb.sort_values('AdjPts')
qb04_top = final_04qb.iloc[-1]
qb04_top = qb04_top.name
final_04wr = df04wr[['Player','Pos','Tm','AdjPts']]
final_04wr = final_04wr.groupby(['Player']).sum()
final_04wr = final_04wr.sort_values('AdjPts')
wr04_top = final_04wr.iloc[-1]
wr04_top = wr04_top.name
final_04te = df04te[['Player','Pos','Tm','AdjPts']]
final_04te = final_04te.groupby(['Player']).sum()
final_04te = final_04te.sort_values('AdjPts')
te04_top = final_04te.iloc[-1]
te04_top = te04_top.name
final_04fb = df04fb[['Player','Pos','Tm','AdjPts']]
final_04fb = final_04fb.groupby(['Player']).sum()
final_04fb = final_04fb.sort_values('AdjPts')
fb04_top = final_04fb.iloc[-1]
fb04_top = fb04_top.name
final_04hb = df04hb[['Player','Pos','Tm','AdjPts']]
final_04hb = final_04hb.groupby(['Player']).sum()
final_04hb = final_04hb.sort_values('AdjPts')
hb04_top = final_04hb.iloc[-1]
hb04_top = hb04_top.name
final_05qb = df05qb[['Player','Pos','Tm','AdjPts']]
final_05qb = final_05qb.groupby(['Player']).sum()
final_05qb = final_05qb.sort_values('AdjPts')
qb05_top = final_05qb.iloc[-1]
qb05_top = qb05_top.name
final_05wr = df05wr[['Player','Pos','Tm','AdjPts']]
final_05wr = final_05wr.groupby(['Player']).sum()
final_05wr = final_05wr.sort_values('AdjPts')
wr05_top = final_05wr.iloc[-1]
wr05_top = wr05_top.name
final_05te = df05te[['Player','Pos','Tm','AdjPts']]
final_05te = final_05te.groupby(['Player']).sum()
final_05te = final_05te.sort_values('AdjPts')
te05_top = final_05te.iloc[-1]
te05_top = te05_top.name
final_05fb = df05fb[['Player','Pos','Tm','AdjPts']]
final_05fb = final_05fb.groupby(['Player']).sum()
final_05fb = final_05fb.sort_values('AdjPts')
fb05_top = final_05fb.iloc[-1]
fb05_top = fb05_top.name
final_05hb = df05hb[['Player','Pos','Tm','AdjPts']]
final_05hb = final_05hb.groupby(['Player']).sum()
final_05hb = final_05hb.sort_values('AdjPts')
hb05_top = final_05hb.iloc[-1]
hb05_top = hb05_top.name
final_06qb = df06qb[['Player','Pos','Tm','AdjPts']]
final_06qb = final_06qb.groupby(['Player']).sum()
final_06qb = final_06qb.sort_values('AdjPts')
qb06_top = final_06qb.iloc[-1]
qb06_top = qb06_top.name
final_06wr = df06wr[['Player','Pos','Tm','AdjPts']]
final_06wr = final_06wr.groupby(['Player']).sum()
final_06wr = final_06wr.sort_values('AdjPts')
wr06_top = final_06wr.iloc[-1]
wr06_top = wr06_top.name
final_06te = df06te[['Player','Pos','Tm','AdjPts']]
final_06te = final_06te.groupby(['Player']).sum()
final_06te = final_06te.sort_values('AdjPts')
te06_top = final_06te.iloc[-1]
te06_top = te06_top.name
final_06fb = df06fb[['Player','Pos','Tm','AdjPts']]
final_06fb = final_06fb.groupby(['Player']).sum()
final_06fb = final_06fb.sort_values('AdjPts')
fb06_top = final_06fb.iloc[-1]
fb06_top = fb06_top.name
final_06hb = df06hb[['Player','Pos','Tm','AdjPts']]
final_06hb = final_06hb.groupby(['Player']).sum()
final_06hb = final_06hb.sort_values('AdjPts')
hb06_top = final_06hb.iloc[-1]
hb06_top = hb06_top.name
final_07qb = df07qb[['Player','Pos','Tm','AdjPts']]
final_07qb = final_07qb.groupby(['Player']).sum()
final_07qb = final_07qb.sort_values('AdjPts')
qb07_top = final_07qb.iloc[-1]
qb07_top = qb07_top.name
final_07wr = df07wr[['Player','Pos','Tm','AdjPts']]
final_07wr = final_07wr.groupby(['Player']).sum()
final_07wr = final_07wr.sort_values('AdjPts')
wr07_top = final_07wr.iloc[-1]
wr07_top = wr07_top.name
final_07te = df07te[['Player','Pos','Tm','AdjPts']]
final_07te = final_07te.groupby(['Player']).sum()
final_07te = final_07te.sort_values('AdjPts')
te07_top = final_07te.iloc[-1]
te07_top = te07_top.name
final_07fb = df07fb[['Player','Pos','Tm','AdjPts']]
final_07fb = final_07fb.groupby(['Player']).sum()
final_07fb = final_07fb.sort_values('AdjPts')
fb07_top = final_07fb.iloc[-1]
fb07_top = fb07_top.name
final_07hb = df07hb[['Player','Pos','Tm','AdjPts']]
final_07hb = final_07hb.groupby(['Player']).sum()
final_07hb = final_07hb.sort_values('AdjPts')
hb07_top = final_07hb.iloc[-1]
hb07_top = hb07_top.name
final_08qb = df08qb[['Player','Pos','Tm','AdjPts']]
final_08qb = final_08qb.groupby(['Player']).sum()
final_08qb = final_08qb.sort_values('AdjPts')
qb08_top = final_08qb.iloc[-1]
qb08_top = qb08_top.name
final_08wr = df08wr[['Player','Pos','Tm','AdjPts']]
final_08wr = final_08wr.groupby(['Player']).sum()
final_08wr = final_08wr.sort_values('AdjPts')
wr08_top = final_08wr.iloc[-1]
wr08_top = wr08_top.name
final_08te = df08te[['Player','Pos','Tm','AdjPts']]
final_08te = final_08te.groupby(['Player']).sum()
final_08te = final_08te.sort_values('AdjPts')
te08_top = final_08te.iloc[-1]
te08_top = te08_top.name
final_08fb = df08fb[['Player','Pos','Tm','AdjPts']]
final_08fb = final_08fb.groupby(['Player']).sum()
final_08fb = final_08fb.sort_values('AdjPts')
fb08_top = final_08fb.iloc[-1]
fb08_top = fb08_top.name
final_08hb = df08hb[['Player','Pos','Tm','AdjPts']]
final_08hb = final_08hb.groupby(['Player']).sum()
final_08hb = final_08hb.sort_values('AdjPts')
hb08_top = final_08hb.iloc[-1]
hb08_top = hb08_top.name
final_09qb = df09qb[['Player','Pos','Tm','AdjPts']]
final_09qb = final_09qb.groupby(['Player']).sum()
final_09qb = final_09qb.sort_values('AdjPts')
qb09_top = final_09qb.iloc[-1]
qb09_top = qb09_top.name
final_09wr = df09wr[['Player','Pos','Tm','AdjPts']]
final_09wr = final_09wr.groupby(['Player']).sum()
final_09wr = final_09wr.sort_values('AdjPts')
wr09_top = final_09wr.iloc[-1]
wr09_top = wr09_top.name
final_09te = df09te[['Player','Pos','Tm','AdjPts']]
final_09te = final_09te.groupby(['Player']).sum()
final_09te = final_09te.sort_values('AdjPts')
te09_top = final_09te.iloc[-1]
te09_top = te09_top.name
final_09fb = df09fb[['Player','Pos','Tm','AdjPts']]
final_09fb = final_09fb.groupby(['Player']).sum()
final_09fb = final_09fb.sort_values('AdjPts')
fb09_top = final_09fb.iloc[-1]
fb09_top = fb09_top.name
final_09hb = df09hb[['Player','Pos','Tm','AdjPts']]
final_09hb = final_09hb.groupby(['Player']).sum()
final_09hb = final_09hb.sort_values('AdjPts')
hb09_top = final_09hb.iloc[-1]
hb09_top = hb09_top.name
final_10qb = df10qb[['Player','Pos','Tm','AdjPts']]
final_10qb = final_10qb.groupby(['Player']).sum()
final_10qb = final_10qb.sort_values('AdjPts')
qb10_top = final_10qb.iloc[-1]
qb10_top = qb10_top.name
final_10wr = df10wr[['Player','Pos','Tm','AdjPts']]
final_10wr = final_10wr.groupby(['Player']).sum()
final_10wr = final_10wr.sort_values('AdjPts')
wr10_top = final_10wr.iloc[-1]
wr10_top = wr10_top.name
final_10te = df10te[['Player','Pos','Tm','AdjPts']]
final_10te = final_10te.groupby(['Player']).sum()
final_10te = final_10te.sort_values('AdjPts')
te10_top = final_10te.iloc[-1]
te10_top = te10_top.name
final_10fb = df10fb[['Player','Pos','Tm','AdjPts']]
final_10fb = final_10fb.groupby(['Player']).sum()
final_10fb = final_10fb.sort_values('AdjPts')
fb10_top = final_10fb.iloc[-1]
fb10_top = fb10_top.name
final_10hb = df10hb[['Player','Pos','Tm','AdjPts']]
final_10hb = final_10hb.groupby(['Player']).sum()
final_10hb = final_10hb.sort_values('AdjPts')
hb10_top = final_10hb.iloc[-1]
hb10_top = hb10_top.name
final_11qb = df11qb[['Player','Pos','Tm','AdjPts']]
final_11qb = final_11qb.groupby(['Player']).sum()
final_11qb = final_11qb.sort_values('AdjPts')
qb11_top = final_11qb.iloc[-1]
qb11_top = qb11_top.name
final_11wr = df11wr[['Player','Pos','Tm','AdjPts']]
final_11wr = final_11wr.groupby(['Player']).sum()
final_11wr = final_11wr.sort_values('AdjPts')
wr11_top = final_11wr.iloc[-1]
wr11_top = wr11_top.name
final_11te = df11te[['Player','Pos','Tm','AdjPts']]
final_11te = final_11te.groupby(['Player']).sum()
final_11te = final_11te.sort_values('AdjPts')
te11_top = final_11te.iloc[-1]
te11_top = te11_top.name
final_11fb = df11fb[['Player','Pos','Tm','AdjPts']]
final_11fb = final_11fb.groupby(['Player']).sum()
final_11fb = final_11fb.sort_values('AdjPts')
fb11_top = final_11fb.iloc[-1]
fb11_top = fb11_top.name
final_11hb = df11hb[['Player','Pos','Tm','AdjPts']]
final_11hb = final_11hb.groupby(['Player']).sum()
final_11hb = final_11hb.sort_values('AdjPts')
hb11_top = final_11hb.iloc[-1]
hb11_top = hb11_top.name
final_12qb = df12qb[['Player','Pos','Tm','AdjPts']]
final_12qb = final_12qb.groupby(['Player']).sum()
final_12qb = final_12qb.sort_values('AdjPts')
qb12_top = final_12qb.iloc[-1]
qb12_top = qb12_top.name
final_12wr = df12wr[['Player','Pos','Tm','AdjPts']]
final_12wr = final_12wr.groupby(['Player']).sum()
final_12wr = final_12wr.sort_values('AdjPts')
wr12_top = final_12wr.iloc[-1]
wr12_top = wr12_top.name
final_12te = df12te[['Player','Pos','Tm','AdjPts']]
final_12te = final_12te.groupby(['Player']).sum()
final_12te = final_12te.sort_values('AdjPts')
te12_top = final_12te.iloc[-1]
te12_top = te12_top.name
final_12fb = df12fb[['Player','Pos','Tm','AdjPts']]
final_12fb = final_12fb.groupby(['Player']).sum()
final_12fb = final_12fb.sort_values('AdjPts')
fb12_top = final_12fb.iloc[-1]
fb12_top = fb12_top.name
final_12hb = df12hb[['Player','Pos','Tm','AdjPts']]
final_12hb = final_12hb.groupby(['Player']).sum()
final_12hb = final_12hb.sort_values('AdjPts')
hb12_top = final_12hb.iloc[-1]
hb12_top = hb12_top.name
final_13qb = df13qb[['Player','Pos','Tm','AdjPts']]
final_13qb = final_13qb.groupby(['Player']).sum()
final_13qb = final_13qb.sort_values('AdjPts')
qb13_top = final_13qb.iloc[-1]
qb13_top = qb13_top.name
final_13wr = df13wr[['Player','Pos','Tm','AdjPts']]
final_13wr = final_13wr.groupby(['Player']).sum()
final_13wr = final_13wr.sort_values('AdjPts')
wr13_top = final_13wr.iloc[-1]
wr13_top = wr13_top.name
final_13te = df13te[['Player','Pos','Tm','AdjPts']]
final_13te = final_13te.groupby(['Player']).sum()
final_13te = final_13te.sort_values('AdjPts')
te13_top = final_13te.iloc[-1]
te13_top = te13_top.name
final_13fb = df13fb[['Player','Pos','Tm','AdjPts']]
final_13fb = final_13fb.groupby(['Player']).sum()
final_13fb = final_13fb.sort_values('AdjPts')
fb13_top = final_13fb.iloc[-1]
fb13_top = fb13_top.name
final_13hb = df13hb[['Player','Pos','Tm','AdjPts']]
final_13hb = final_13hb.groupby(['Player']).sum()
final_13hb = final_13hb.sort_values('AdjPts')
hb13_top = final_13hb.iloc[-1]
hb13_top = hb13_top.name
final_14qb = df14qb[['Player','Pos','Tm','AdjPts']]
final_14qb = final_14qb.groupby(['Player']).sum()
final_14qb = final_14qb.sort_values('AdjPts')
qb14_top = final_14qb.iloc[-1]
qb14_top = qb14_top.name
final_14wr = df14wr[['Player','Pos','Tm','AdjPts']]
final_14wr = final_14wr.groupby(['Player']).sum()
final_14wr = final_14wr.sort_values('AdjPts')
wr14_top = final_14wr.iloc[-1]
wr14_top = wr14_top.name
final_14te = df14te[['Player','Pos','Tm','AdjPts']]
final_14te = final_14te.groupby(['Player']).sum()
final_14te = final_14te.sort_values('AdjPts')
te14_top = final_14te.iloc[-1]
te14_top = te14_top.name
final_14fb = df14fb[['Player','Pos','Tm','AdjPts']]
final_14fb = final_14fb.groupby(['Player']).sum()
final_14fb = final_14fb.sort_values('AdjPts')
fb14_top = final_14fb.iloc[-1]
fb14_top = fb14_top.name
final_14hb = df14hb[['Player','Pos','Tm','AdjPts']]
final_14hb = final_14hb.groupby(['Player']).sum()
final_14hb = final_14hb.sort_values('AdjPts')
hb14_top = final_14hb.iloc[-1]
hb14_top = hb14_top.name
final_15qb = df15qb[['Player','Pos','Tm','AdjPts']]
final_15qb = final_15qb.groupby(['Player']).sum()
final_15qb = final_15qb.sort_values('AdjPts')
qb15_top = final_15qb.iloc[-1]
qb15_top = qb15_top.name
final_15wr = df15wr[['Player','Pos','Tm','AdjPts']]
final_15wr = final_15wr.groupby(['Player']).sum()
final_15wr = final_15wr.sort_values('AdjPts')
wr15_top = final_15wr.iloc[-1]
wr15_top = wr15_top.name
final_15te = df15te[['Player','Pos','Tm','AdjPts']]
final_15te = final_15te.groupby(['Player']).sum()
final_15te = final_15te.sort_values('AdjPts')
te15_top = final_15te.iloc[-1]
te15_top = te15_top.name
final_15fb = df15fb[['Player','Pos','Tm','AdjPts']]
final_15fb = final_15fb.groupby(['Player']).sum()
final_15fb = final_15fb.sort_values('AdjPts')
fb15_top = final_15fb.iloc[-1]
fb15_top = fb15_top.name
final_15hb = df15hb[['Player','Pos','Tm','AdjPts']]
final_15hb = final_15hb.groupby(['Player']).sum()
final_15hb = final_15hb.sort_values('AdjPts')
hb15_top = final_15hb.iloc[-1]
hb15_top = hb15_top.name
final_16qb = df16qb[['Player','Pos','Tm','AdjPts']]
final_16qb = final_16qb.groupby(['Player']).sum()
final_16qb = final_16qb.sort_values('AdjPts')
qb16_top = final_16qb.iloc[-1]
qb16_top = qb16_top.name
final_16wr = df16wr[['Player','Pos','Tm','AdjPts']]
final_16wr = final_16wr.groupby(['Player']).sum()
final_16wr = final_16wr.sort_values('AdjPts')
wr16_top = final_16wr.iloc[-1]
wr16_top = wr16_top.name
final_16te = df16te[['Player','Pos','Tm','AdjPts']]
final_16te = final_16te.groupby(['Player']).sum()
final_16te = final_16te.sort_values('AdjPts')
te16_top = final_16te.iloc[-1]
te16_top = te16_top.name
final_16fb = df16fb[['Player','Pos','Tm','AdjPts']]
final_16fb = final_16fb.groupby(['Player']).sum()
final_16fb = final_16fb.sort_values('AdjPts')
fb16_top = final_16fb.iloc[-1]
fb16_top = fb16_top.name
final_16hb = df16hb[['Player','Pos','Tm','AdjPts']]
final_16hb = final_16hb.groupby(['Player']).sum()
final_16hb = final_16hb.sort_values('AdjPts')
hb16_top = final_16hb.iloc[-1]
hb16_top = hb16_top.name
final_17qb = df17qb[['Player','Pos','Tm','AdjPts']]
final_17qb = final_17qb.groupby(['Player']).sum()
final_17qb = final_17qb.sort_values('AdjPts')
qb17_top = final_17qb.iloc[-1]
qb17_top = qb17_top.name
final_17wr = df17wr[['Player','Pos','Tm','AdjPts']]
final_17wr = final_17wr.groupby(['Player']).sum()
final_17wr = final_17wr.sort_values('AdjPts')
wr17_top = final_17wr.iloc[-1]
wr17_top = wr17_top.name
final_17te = df17te[['Player','Pos','Tm','AdjPts']]
final_17te = final_17te.groupby(['Player']).sum()
final_17te = final_17te.sort_values('AdjPts')
te17_top = final_17te.iloc[-1]
te17_top = te17_top.name
final_17fb = df17fb[['Player','Pos','Tm','AdjPts']]
final_17fb = final_17fb.groupby(['Player']).sum()
final_17fb = final_17fb.sort_values('AdjPts')
fb17_top = final_17fb.iloc[-1]
fb17_top = fb17_top.name
final_17hb = df17hb[['Player','Pos','Tm','AdjPts']]
final_17hb = final_17hb.groupby(['Player']).sum()
final_17hb = final_17hb.sort_values('AdjPts')
hb17_top = final_17hb.iloc[-1]
hb17_top = hb17_top.name
final_18qb = df18qb[['Player','Pos','Tm','AdjPts']]
final_18qb = final_18qb.groupby(['Player']).sum()
final_18qb = final_18qb.sort_values('AdjPts')
qb18_top = final_18qb.iloc[-1]
qb18_top = qb18_top.name
final_18wr = df18wr[['Player','Pos','Tm','AdjPts']]
final_18wr = final_18wr.groupby(['Player']).sum()
final_18wr = final_18wr.sort_values('AdjPts')
wr18_top = final_18wr.iloc[-1]
wr18_top = wr18_top.name
final_18te = df18te[['Player','Pos','Tm','AdjPts']]
final_18te = final_18te.groupby(['Player']).sum()
final_18te = final_18te.sort_values('AdjPts')
te18_top = final_18te.iloc[-1]
te18_top = te18_top.name
final_18fb = df18fb[['Player','Pos','Tm','AdjPts']]
final_18fb = final_18fb.groupby(['Player']).sum()
final_18fb = final_18fb.sort_values('AdjPts')
fb18_top = final_18fb.iloc[-1]
fb18_top = fb18_top.name
final_18hb = df18hb[['Player','Pos','Tm','AdjPts']]
final_18hb = final_18hb.groupby(['Player']).sum()
final_18hb = final_18hb.sort_values('AdjPts')
hb18_top = final_18hb.iloc[-1]
hb18_top = hb18_top.name
final_19qb = df19qb[['Player','Pos','Tm','AdjPts']]
final_19qb = final_19qb.groupby(['Player']).sum()
final_19qb = final_19qb.sort_values('AdjPts')
qb19_top = final_19qb.iloc[-1]
qb19_top = qb19_top.name
final_19wr = df19wr[['Player','Pos','Tm','AdjPts']]
final_19wr = final_19wr.groupby(['Player']).sum()
final_19wr = final_19wr.sort_values('AdjPts')
wr19_top = final_19wr.iloc[-1]
wr19_top = wr19_top.name
final_19te = df19te[['Player','Pos','Tm','AdjPts']]
final_19te = final_19te.groupby(['Player']).sum()
final_19te = final_19te.sort_values('AdjPts')
te19_top = final_19te.iloc[-1]
te19_top = te19_top.name
final_19fb = df19fb[['Player','Pos','Tm','AdjPts']]
final_19fb = final_19fb.groupby(['Player']).sum()
final_19fb = final_19fb.sort_values('AdjPts')
fb19_top = final_19fb.iloc[-1]
fb19_top = fb19_top.name
final_19hb = df19hb[['Player','Pos','Tm','AdjPts']]
final_19hb = final_19hb.groupby(['Player']).sum()
final_19hb = final_19hb.sort_values('AdjPts')
hb19_top = final_19hb.iloc[-1]
hb19_top = hb19_top.name

topplayers_data = {
    'Year': [1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019],
    'QB': [qb99_top,qb00_top,qb01_top,qb02_top,qb03_top,qb04_top,qb05_top,qb06_top,qb07_top,qb08_top,qb09_top,qb10_top,qb11_top,qb12_top,qb13_top,qb14_top,qb15_top,qb16_top,qb17_top,qb18_top,qb19_top],
    'WR': [wr99_top,wr00_top,wr01_top,wr02_top,wr03_top,wr04_top,wr05_top,wr06_top,wr07_top,wr08_top,wr09_top,wr10_top,wr11_top,wr12_top,wr13_top,wr14_top,wr15_top,wr16_top,wr17_top,wr18_top,wr19_top],
    'TE': [te99_top,te00_top,te01_top,te02_top,te03_top,te04_top,te05_top,te06_top,te07_top,te08_top,te09_top,te10_top,te11_top,te12_top,te13_top,te14_top,te15_top,te16_top,te17_top,te18_top,te19_top],
    'FB': [fb99_top,fb00_top,fb01_top,fb02_top,fb03_top,fb04_top,fb05_top,fb06_top,fb07_top,fb08_top,fb09_top,fb10_top,fb11_top,fb12_top,fb13_top,fb14_top,fb15_top,fb16_top,fb17_top,fb18_top,fb19_top],
    'HB': [hb99_top,hb00_top,hb01_top,hb02_top,hb03_top,hb04_top,hb05_top,hb06_top,hb07_top,hb08_top,hb09_top,hb10_top,hb11_top,hb12_top,hb13_top,hb14_top,hb15_top,hb16_top,hb17_top,hb18_top,hb19_top],
}
topplayers = pd.DataFrame(data=topplayers_data)
pd.set_option('display.max_columns', None)

# Results

The yearly top players per position:

In [ ]:
print(topplayers)

    Year                QB                WR              TE               FB  \
0   1999       Kurt Warner        Randy Moss    Wesley Walls  Jonathan Linton   
1   2000  Daunte Culpepper       Cris Carter      Chad Lewis  Tony Richardson   
2   2001       Kurt Warner      David Boston   Tony Gonzalez    Larry Centers   
3   2002       Rich Gannon        Hines Ward    Bubba Franks    Larry Centers   
4   2003  Daunte Culpepper        Randy Moss   Tony Gonzalez  Richie Anderson   
5   2004  Daunte Culpepper        Torry Holt   Alge Crumpler     Patrick Pass   
6   2005         Tom Brady  Larry Fitzgerald   Antonio Gates     Kyle Johnson   
7   2006    Peyton Manning     Terrell Owens   Antonio Gates     Mike Alstott   
8   2007         Tom Brady        Randy Moss    Chris Cooley   Leonard Weaver   
9   2008        Drew Brees       Steve Smith   Tony Gonzalez   Le'Ron McClain   
10  2009     Aaron Rodgers       Steve Smith     Brent Celek   Le'Ron McClain   
11  2010         Tom Brady  